Даний блокнот необхідний для виконання прогнозів на вже навченій моделі, що збережена в файлі model.pkl.
Дані, що будуть використовуватись для пронозу мають бути в також форматі, що і ті, що використовувались для навчання - тобто це має бути csv файл з усіма фічами. Далі виконуються потрібні для нашої моделі перетворення вхідних даних і виконується прогноз.

### **Обгрунтування вибору основної метрики**

У якості основної метрики ми вирішили обрати **MAE**. Для цього краще розповімо чому нам не підійшли інші.

Метрика MAPE, що розраховується за формулою $MAPE=\frac{1}{n}\times \left( \frac{{{y}_{pred}}-{{y}_{true}}}{{{y}_{true}}} \right)$, містить в знаменику значення цільвої змінної, але в нашому наборі цільова змінна досить часто близька до нуля і тому при ділені на таке число ми отримуємо достатньо великі значення частки.

Метрика RMSE досить сильно штрафує великі помилки. Тобто, наприклад, якщо майже на всіх даних у нас буде незначна помилка, але на якомусь із прикладів матиме велике значення, то RMSE також значно збільшиться. Дана метрика дуже чутлива до таких викидів в значеннях помилок. Оскільки для важлива загальна якість моделі і ми не приділяли досить багато уваги до конкретних прикладів, то нам не є дуже важливим контроль того, що на всіх прикладах мала помилка і лише на деяких вона значно більша. Тому дана метрика нам не дуже підходить. Формула для розрахунку - $RMSE=\sqrt{\frac{1}{n}\times {{\left( {{y}_{pred}}-{{y}_{true}} \right)}^{2}}}$

Саме тому ми обрали метрику MAE - вона в достатній мірі відображає помилку на більшості прикладів та проста в інтерпретації. Розраховується за формулою - $MAE=\frac{1}{n}\times \left| {{y}_{pred}}-{{y}_{true}} \right|$

### **Завантажуємо збережену модель з файлу**

In [1]:
import pickle
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

### **Завантажуємо вхідні дані для прогнозу**

In [2]:
import pandas as pd
import numpy as np
#data = pd.read_csv(input("Шлях до набору даних:"), nrows=10).fillna(0)
data = pd.read_csv(input("Шлях до набору даних:")).fillna(0)
print(f'Всього спостережень в наборі: {data.shape[0]}')

Шлях до набору даних: TEST TASK (dataset).csv


Всього спостережень в наборі: 10


### **Перетворюємо вхідні дані до потрібного для моделі вигляду**

In [4]:
def trend_coeff(observations):
    X = np.array([0, 1, 3, 7])[:len(observations), np.newaxis]
    model = LinearRegression(n_jobs=-1).fit(X, np.array(observations)[:, np.newaxis])
    return [model.coef_[0][0], model.intercept_[0]]

# Sessions
data[['total_sessions_trend', 'total_sessions_bias']] = data[[f'total_sessions_day{day}' for day in [0, 1, 3, 7]]].\
    apply(trend_coeff, axis=1).apply(pd.Series).rename(columns={0: 'total_sessions_trend', 1: 'total_sessions_bias'})

# Chapters
for idx in [1, 3, 9]:
    data[f'chapters_finished_ratio{idx}'] = (
        (data[f'chapters_finished_session{idx}'] - data[f'chapters_closed_session{idx}']) / data[f'chapters_opened_session{idx}']
    ).fillna(0)
data['chapters_finished_ratio'] = (data['chapters_finished_ratio1'] + data['chapters_finished_ratio3'] + data['chapters_finished_ratio9']) / 3

# LTV
for idx in [0, 1, 3]:
    data[f'full_ltv_day{idx}'] = data[f'app_sub_ltv_day0'] + data[f'app_iap_ltv_day{idx}'] + data[f'ad_ltv_day{idx}']
data['mean_full_ltv'] = (data['full_ltv_day0'] + data['full_ltv_day1'] / 2 + data['full_ltv_day3'] / 4) / 3

# Tickets
data[['tickets_spent_trend', 'tickets_spent_bias']] = data[[f'tickets_spent_day{day}' for day in [0, 1, 3, 7]]].\
    apply(trend_coeff, axis=1).apply(pd.Series).rename(columns={0: 'tickets_spent_trend', 1: 'tickets_spent_bias'})

# Diamonds
data[['diamonds_spent_trend', 'diamonds_spent_bias']] = data[[f'diamonds_spent_session{day}' for day in [1, 3, 9]]].\
    apply(trend_coeff, axis=1).apply(pd.Series).rename(columns={0: 'diamonds_spent_trend', 1: 'diamonds_spent_bias'})

data_new = data[[
    'total_sessions_trend', 'total_sessions_bias', 'chapters_finished_ratio', 'mean_full_ltv', 'tickets_spent_trend', 'tickets_spent_bias', 
    'diamonds_spent_trend', 'diamonds_spent_bias'#, 'target_full_ltv_day30'
]].copy()

X = data_new.loc[:, data_new.columns != 'target_full_ltv_day30']
#y = data_new['target_full_ltv_day30']

### **Отримуємо прогнозні значення**

In [6]:
#from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
with pd.option_context('display.max_rows', None):
    display(pd.DataFrame(data=model.predict(X), columns=['Predicted full LTV value']))
#print(mean_absolute_percentage_error(model.predict(X), y))
#print(mean_squared_error(model.predict(X), y))
#print(mean_absolute_error(model.predict(X), y))

,Predicted full LTV value
0,0.003465
1,0.003465
2,0.003465
3,0.003465
4,0.003465
5,0.003465
6,0.003465
7,0.003465
8,0.003465
9,0.003465
